In [1]:
import os
import pathlib
import sys

default_dir_path = str(pathlib.Path(os.getcwd()).parent.absolute())

sys.path.append(default_dir_path)

from properties import ApplicationProperties

[2020-11-12 08:00:22,358][INFO][properties.py:22] Complete to apply the random seed, RANDOM_SEED : 777


In [2]:
from torch import optim
from torch.nn import functional as F
import torch

from dataset.factory import DatasetFactory
from domain.hyperparameters import Hyperparameters
from logger import logger
from model.factory import ModelFactory
from trainer.factory import TrainerFactory

In [3]:
model_name = "cnn_custom"
data_name = "kaggle_casting_data"

In [4]:
# Dataset Controller
dataset_factory = DatasetFactory.create(data_name=data_name)
train_dataset, val_dataset, test_dataset \
    = dataset_factory.train_dataset, dataset_factory.val_dataset, dataset_factory.test_dataset
train_dataloader, val_dataloader, test_dataloader \
    = dataset_factory.train_dataloader, dataset_factory.val_dataloader, dataset_factory.test_dataloader

[2020-11-12 08:00:22,818][INFO][factory.py:72] Data selected : kaggle_casting_data


In [5]:
# Model Controller
model_factory = ModelFactory.create(model_name=model_name)
model = model_factory.model

[2020-11-12 08:00:22,882][INFO][factory.py:24] Model selected : cnn_custom


In [6]:
# Trainer Controller
trainer_factory = TrainerFactory.create(
    model_name=model_name,
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    tqdm_env="jupyter"
)

[2020-11-12 08:00:25,965][INFO][base.py:39] Model set to cuda


In [7]:
trainer_factory.trainer.hyperparameters = Hyperparameters(
    optimizer_cls=optim.Adam,
    criterion=F.binary_cross_entropy,
    n_epoch=5,
    lr=1e-3,
    hypothesis_threshold=0.5,
    weight_decay=0,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

## Training

In [8]:
result_dict = trainer_factory.do(mode="train")

[2020-11-12 07:52:22,553][INFO][factory.py:61] Start to train


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/workspace/sy/byeonggil/MDPI_2020_remote/trainer/cnn_custom_trainer.py:55: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  loss = criterion(hypothesis, label_batch)
/workspace/sy/byeonggil/MDPI_2020_remote/trainer/cnn_custom_trainer.py:55: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])) is deprecated. Please ensure they have the same size.
  loss = criterion(hypothesis, label_batch)
/workspace/sy/byeonggil/MDPI_2020_remote/trainer/cnn_custom_trainer.py:150: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please 

[Epoch - 0]
Train - Accuracy : 0.4328358, Precision : 0.4328132, Recall : 0.9937391, F1 : 0.602997
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)
[Epoch - 1]
Train - Accuracy : 0.4334389, Precision : 0.4334389, Recall : 1.0, F1 : 0.6047539
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)
[Epoch - 2]
Train - Accuracy : 0.4334389, Precision : 0.4334389, Recall : 1.0, F1 : 0.6047539
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)
[Epoch - 3]
Train - Accuracy : 0.4334389, Precision : 0.4334389, Recall : 1.0, F1 : 0.6047539
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356

[2020-11-12 07:53:52,335][INFO][base.py:59] Succeed to load best model, device: cpu
[2020-11-12 07:53:52,336][INFO][factory.py:69] Completed to train


[Epoch - 4]
Train - Accuracy : 0.4334389, Precision : 0.4334389, Recall : 1.0, F1 : 0.6047539
Val - Accuracy : 0.3664336, Precision : 0.3664336, Recall : 1.0, F1 : 0.5363357
Best Accuracy : 0.36643356643356645 (epoch : 0)
Best F1 : 0.5363357215967247 (epoch : 0)



## Predict

In [8]:
trainer_factory.trainer.load_best_model()

[2020-11-12 08:00:26,014][INFO][base.py:60] Succeed to load best model, device: cuda


In [9]:
result_dict = trainer_factory.do(mode="predict")

[2020-11-12 08:00:26,062][INFO][factory.py:61] Start to predict
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/workspace/sy/byeonggil/MDPI_2020_remote/trainer/cnn_custom_trainer.py:150: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  loss = criterion(hypothesis, label_batch)
/workspace/sy/byeonggil/MDPI_2020_remote/trainer/cnn_custom_trainer.py:150: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])) is deprecated. Please ensure they have the same size.
  loss = criterion(hypothesis, label_batch)
[2020-11-12 08:00:27,657][INFO][factory.py:69] Completed to predict


In [10]:
result_dict

{'loss': 17.006593393242877,
 'accuracy': 0.36643356643356645,
 'precision': 0.36643356643356645,
 'recall': 1.0,
 'f1': 0.5363357215967247}